In [1]:
# BAR CHART 

In [3]:
# with trend lines

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import socket

# Function to find a free port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]

# Load dataset
df = pd.read_csv('./df_ca2_dataviz.csv')

# Initialize Dash app
app = dash.Dash(__name__)

# Function to convert input to sentence case
def to_sentence_case(name):
    return name.capitalize()

# App layout
app.layout = html.Div([
    html.Label('Enter a Name:'),
    dcc.Input(id='name-input', type='text', placeholder='Enter a name'),
    html.Br(),
    html.Div([
        dcc.Graph(id='trend-graph-usa-ireland'),
    ])
])

# Callback to update the graph based on user input
@app.callback(
    Output('trend-graph-usa-ireland', 'figure'),
    [Input('name-input', 'value')]
)
def update_graph(selected_name):
    if selected_name is None:
        selected_name = ''  # If no name is selected, use an empty string

    selected_name = to_sentence_case(selected_name)

    filtered_df = df[df['Name'] == selected_name]

    # Create a subplot with shared x-axis and separate y-axes
    fig = make_subplots(specs=[[{'secondary_y': True}]])

    # Add bar trace for the USA with trend line
    fig.add_trace(
        go.Bar(x=filtered_df[filtered_df['Country'] == 'USA']['Year'],
               y=filtered_df[filtered_df['Country'] == 'USA']['Count'],
               name='USA Count',
               marker=dict(color='rgba(0, 0, 255, 0.6)'),
               showlegend=False),  # Use blue color for the USA

        secondary_y=False)  # Use the primary y-axis

    # Add trend line for the USA
    fig.add_trace(
        go.Scatter(x=filtered_df[filtered_df['Country'] == 'USA']['Year'],
                   y=filtered_df[filtered_df['Country'] == 'USA']['Count'].rolling(window=3).mean(),
                   mode='lines',
                   line=dict(color='blue'),
                   name='USA Trend',
                   showlegend=True),
        secondary_y=False)  # Use the primary y-axis

    # Add bar trace for Ireland with trend line
    fig.add_trace(
        go.Bar(x=filtered_df[filtered_df['Country'] == 'Ireland']['Year'],
               y=filtered_df[filtered_df['Country'] == 'Ireland']['Count'],
               name='Irish Count',
               marker=dict(color='rgba(0, 128, 0, 0.6)'),
               showlegend=False),  # Use green color for Ireland

        secondary_y=True)  # Use the secondary y-axis

    # Add trend line for Ireland
    fig.add_trace(
        go.Scatter(x=filtered_df[filtered_df['Country'] == 'Ireland']['Year'],
                   y=filtered_df[filtered_df['Country'] == 'Ireland']['Count'].rolling(window=3).mean(),
                   mode='lines',
                   line=dict(color='green'),
                   name='Irish Trend',
                   showlegend=True),
        secondary_y=True)  # Use the secondary y-axis

    # Update layout
    fig.update_layout(
        barmode='group',
        xaxis=dict(title='Year'),
        yaxis=dict(title='USA Count', side='left'),
        yaxis2=dict(title='Irish Count', overlaying='y', side='right', position=0.98),  # Adjust position to place on the right
        legend=dict(x=0.7, y=1.0),  # Adjust legend position
    )

    return fig

# Run the app
if __name__ == '__main__':
    free_port = find_free_port()
    app.run_server(debug=True, port=free_port)
